In [1]:
import sys
import string
import itertools
from collections import Counter, defaultdict
import re

from pathlib import Path
import os

import pandas as pd
import numpy as np

In [12]:
%load_ext line_profiler

In [2]:
data = Path('../data/day_04.txt').read_text()

In [3]:
data = data.split('\n\n')

In [4]:
numbers, *boards = data

In [5]:
numbers = [int(k) for k in numbers.split(',')]

In [6]:
boards = [[[int(k) for k in row.split()] for row in board.splitlines()] for board in boards]

In [47]:
class Board:
    def __init__(self, board):
        self.pos_mapper = {v: (x, y) for (x, row) in enumerate(board) for (y, v) in enumerate(row)}
        self.column_bingos = dict()
        self.row_bingos = dict()
        self.board_size = len(board)
        self.column_max = 0
        self.row_max = 0
    
    def play_number(self, number):

        x, y = self.pos_mapper[number]
        x_update = self.row_bingos.get(x, 0) + 1
        y_update = self.column_bingos.get(y, 0) + 1
        self.column_bingos[y] = y_update
        self.row_bingos[x] = x_update
        if y_update > self.column_max:
            self.column_max = y_update
        if x_update > self.row_max:
            self.row_max = x_update
        # self.column_max = max(self.column_max, y_update)
        # self.row_max = max(self.row_max, x_update)
        
    @property
    def bingo(self):
        return not (self.column_max < self.board_size and self.row_max < self.board_size)
    
    def reset(self):
        self.column_bingos = dict()
        self.row_bingos = dict()
        self.column_max = 0
        self.row_max = 0

In [48]:
def part_a():
    boards_obj_list = [Board(board) for board in boards]

    called = set()

    for number in numbers:
        # print(number)
        called.add(number)
        for board in boards_obj_list:
            if number not in board.pos_mapper:
                continue
            board.play_number(number)
            if board.bingo:
                return sum(board.pos_mapper.keys() - called) * number

print(part_a())
%timeit part_a()

23177
754 µs ± 11.9 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [61]:
%lprun -f part_a -f Board.play_number part_a()

Timer unit: 1e-06 s

Total time: 0.001862 s
File: <ipython-input-47-487b2af5ad75>
Function: play_number at line 10

Line #      Hits         Time  Per Hit   % Time  Line Contents
    10                                               def play_number(self, number):
    11                                           
    12       521        285.0      0.5     15.3          x, y = self.pos_mapper[number]
    13       521        275.0      0.5     14.8          x_update = self.row_bingos.get(x, 0) + 1
    14       521        309.0      0.6     16.6          y_update = self.column_bingos.get(y, 0) + 1
    15       521        208.0      0.4     11.2          self.column_bingos[y] = y_update
    16       521        192.0      0.4     10.3          self.row_bingos[x] = x_update
    17       521        192.0      0.4     10.3          if y_update > self.column_max:
    18       217        109.0      0.5      5.9              self.column_max = y_update
    19       521        189.0      0.4     10.2

In [44]:
%timeit boards_obj_list = [Board(board) for board in boards] # re-initialization required

323 µs ± 4.28 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [59]:
def part_b():
    boards_list = [Board(board) for board in boards] # re-initialization required
    non_bingoed_boards = set(range(len(boards)))
    last_bingo_board = None
    called_numbers = set()

    for number in numbers:

        bingoed_boards = set()
        called_numbers.add(number)
        for board_id in non_bingoed_boards:
            board = boards_list[board_id]
            if number not in board.pos_mapper:
                continue
            board.play_number(number)
            if board.bingo:
                bingoed_boards.add(board_id)
                last_bingo_board = board
        non_bingoed_boards = non_bingoed_boards - bingoed_boards

        if not non_bingoed_boards:
            return sum(last_bingo_board.pos_mapper.keys() - called_numbers) * number

print(part_b())
%timeit part_b()

6804
1.73 ms ± 26.6 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [60]:
%lprun -f part_b part_b()

Timer unit: 1e-06 s

Total time: 0.020427 s
File: <ipython-input-59-f5f3a619c190>
Function: part_b at line 1

Line #      Hits         Time  Per Hit   % Time  Line Contents
     1                                           def part_b():
     2         1        909.0    909.0      4.4      boards_list = [Board(board) for board in boards] # re-initialization required
     3         1          4.0      4.0      0.0      non_bingoed_boards = set(range(len(boards)))
     4         1          1.0      1.0      0.0      last_bingo_board = None
     5         1          1.0      1.0      0.0      called_numbers = set()
     6                                           
     7        87         67.0      0.8      0.3      for number in numbers:
     8                                           
     9        87         53.0      0.6      0.3          bingoed_boards = set()
    10        87         55.0      0.6      0.3          called_numbers.add(number)
    11      6035       3019.0      0.5    